In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM

In [4]:
data = pd.read_csv('dataset_reformate.csv')

data

,pays,date,nombre_total,entree_mensuelle
0,Albanie,2014-02-01,1333,16.0
1,Allemagne,2014-02-01,306716,445.0
2,Andorre,2014-02-01,23,0.0
3,Autriche,2014-02-01,41225,-37.0
4,Bélarus,2014-02-01,1143,39.0
...,...,...,...,...
25795,Total Océanie,2024-10-01,4010,37.0
25796,Apatride,2024-10-01,1254,6.0
25797,Sans nationalité,2024-10-01,272,3.0
25798,Etat inconnu,2024-10-01,1089,2.0


In [5]:
data['date'] = pd.to_datetime(data['date'])  # Conversion en datetime
germany_data = data[data['pays'] == 'Allemagne'].sort_values(by='date')
germany_data

,pays,date,nombre_total,entree_mensuelle
1,Allemagne,2014-02-01,306716,445.0
201,Allemagne,2014-03-01,306689,-27.0
401,Allemagne,2014-04-01,306016,-673.0
601,Allemagne,2014-05-01,306817,801.0
801,Allemagne,2014-06-01,307436,619.0
...,...,...,...,...
24801,Allemagne,2024-06-01,335372,457.0
25001,Allemagne,2024-07-01,335499,127.0
25201,Allemagne,2024-08-01,336162,663.0
25401,Allemagne,2024-09-01,336491,329.0


In [7]:
# Paramètres de séquences temporelles
sequence_length = 12  # Utiliser les 12 derniers mois pour prédire le suivant

# Extraire la variable cible
values = germany_data['entree_mensuelle'].values.reshape(-1, 1)

# Normaliser les données
scaler = MinMaxScaler(feature_range=(0, 1))
normalized_values = scaler.fit_transform(values)

# Créer les séquences temporelles
X, y = [], []
for i in range(len(normalized_values) - sequence_length):
    X.append(normalized_values[i:i + sequence_length])
    y.append(normalized_values[i + sequence_length])

X, y = np.array(X), np.array(y)

# Diviser en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Formes des ensembles
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((93, 12, 1), (24, 12, 1), (93, 1), (24, 1))

In [ ]:
# Définir le modèle LSTM
model = Sequential([
    LSTM(50, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])),
    Dense(1)
])

# Compiler le modèle
model.compile(optimizer='adam', loss='mean_squared_error')

# Entraîner le modèle
history = model.fit(X_train, y_train, epochs=50, batch_size=16, validation_split=0.2, verbose=1)

# Résultat de l'entraînement
history.history